In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
embeddings_df20X = pd.read_hdf('20Xtile_embeding.h5')

In [ ]:
import matplotlib.image as mpimg
def plot_images(image_paths, title):
    fig, axes = plt.subplots(2, 10, figsize=(20, 4))
    fig.suptitle(title, fontsize=16)
    for i, ax in enumerate(axes.flat):
        if i < len(image_paths):
            img = mpimg.imread(image_paths[i])
            ax.imshow(img)
            ax.axis('off')
    plt.show()

In [ ]:
import faiss
vector_20x_df = embeddings_df20X.iloc[:, :1536]
vectors_20x = np.stack(vector_20x_df.values)
d = vectors_20x.shape[1]

In [ ]:
n_clusters = 64
# faiss.normalize_L2(vectors_20x)
kmeans = faiss.Kmeans(d, n_clusters, niter=55, verbose=True, gpu=True, max_points_per_centroid=725970, nredo=3, seed=1234)
kmeans.train(vectors_20x)
print('--Finished Training')
index = faiss.IndexFlatL2(d)
index.add(vectors_20x)
print('Finished Indexing')

In [ ]:
cluster_assignments = kmeans.index.search(vectors_20x, 1)[1]
print('Finished Searching')
embeddings_df20X['cluster'] = cluster_assignments.flatten()
# embeddings_df20X.to_hdf('20Xtile_w_64cluster_fix.h5', key='data')
vector_20x_df = None

In [ ]:
D, I = index.search (kmeans.centroids, 20)
image_dir = './20XTiles/'
cluster_num  = 0
for neighbors in I:
    # print(f"Cluster {cluster_id}: Centroid = {centroids[cluster_id]}")
    print('cluster_num', cluster_num)
    print(f"Nearest 20 Neighbors: {neighbors}\n")
    filenames = []
    for i in neighbors:
        slide_id = embeddings_df20X.iloc[i].ids
        coord1 = int(embeddings_df20X.iloc[i].coord1)
        coord2 = int(embeddings_df20X.iloc[i].coord2)
        filename = image_dir+str(slide_id)+'/'+str(coord1)+'x_'+str(coord2)+'y.jpg'
        filenames.append(filename)
    image_paths = filenames
    plot_images(image_paths, f"Cluster {cluster_num} - Nearest 20 Neighbors")
    cluster_num += 1

In [ ]:
for i in range(n_clusters):
    print('cluster_num', i)
    print(f"Random 20 : ")
    filenames = []
    cluster_df = embeddings_df20X[embeddings_df20X['cluster']==i]
    cluster_df = cluster_df.sample(n=20)
    for row in cluster_df.iterrows():
        row = row[1]
        slide_id = row.ids
        coord1 = int(row.coord1)
        coord2 = int(row.coord2)
        filename = image_dir+str(slide_id)+'/'+str(coord1)+'x_'+str(coord2)+'y.jpg'
        filenames.append(filename)
    image_paths = filenames
    plot_images(image_paths, f"Cluster {i+1} - Random 20")

In [ ]:
vectors_20x = None
embeddings_df20X = None

In [ ]:
# Mannully identified the clusters and add a column of 'is cluster include'
embeddings_df20X = pd.read_hdf('20Xtile_w_64cluster_fix.h5')
cluster_map = pd.read_excel('cluster_is_include.xlsx')
embeddings_df20X = embeddings_df20X.merge(cluster_map, left_on='cluster', right_on='cluster_number', how='left')
# embeddings_df20X.to_hdf('20Xtile_w_64cluster_fix.h5', key='data')


### Draw Umap

In [ ]:
# embeddings_df20X = pd.read_hdf('20Xtile_w_64cluster_fix.h5')
# umap_embeddings = None
# input_embed = embeddings_df20X.sample(frac=0.1, random_state=42)
# for df in [input_embed]:
#     df=df.dropna()
#     # reducer = umap.UMAP(n_neighbors=20, min_dist=0.99, n_components=2, random_state=42, verbose=True)
#     reducer = umap.UMAP(n_neighbors=20, min_dist=0.9, n_components=2, random_state=42, verbose=True)
#     umap_embeddings = reducer.fit_transform(df.iloc[:, :1536], df['cluster'])

In [ ]:
# umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP1', 'UMAP2'])
# umap_df['Label'] = list(input_embed['cluster'])
# umap_df.to_hdf('umap_20x_64cluster.h5', key='data')

In [ ]:
# centroids = np.array([umap_embeddings[input_embed['cluster'] == i].mean(axis=0) for i in range(n_clusters)])

# def plot_umap_fig(umap_embeddings, label, title):
#     umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP1', 'UMAP2'])
#     umap_df['Label'] = list(label)

#     # Plot the UMAP result
#     plt.figure(figsize=(10, 8))
#     scatter = plt.scatter(umap_df['UMAP1'], umap_df['UMAP2'], c=umap_df['Label'], cmap='Spectral', alpha=0.7, s=0.1)

#     # plt.colorbar(scatter, label='Cluster')
#     for i, centroid in enumerate(centroids):
#         plt.text(centroid[0], centroid[1], str(i), fontsize=12, ha='center', va='center', color='black', weight='bold')

#     plt.title(title)
#     plt.xlabel('UMAP1')
#     plt.ylabel('UMAP2')
#     plt.legend(markerscale=30)

#     plt.show() 
# plot_umap_fig(umap_embeddings, input_embed['cluster'], title='20X tile embedding Kmean Umap')